# Import Data

In [1]:
# import modules
import pandas as pd
import glob
import re

In [2]:
# read in all csv files from pos directory
pos_data = glob.glob("data/csv/pos/*.csv")
# print(pos_data)

df_list = []

for file in pos_data:
    chunk = pd.read_csv(file)
    
    cols = {}    # rename df columns and create metada
    for name in chunk.columns[1:]:
        n = name.split('L-')[1]    # remove 'PosMSMSALL/NegMSMSALL'
        cols[name] = n
    chunk = chunk.rename(columns=cols)    # rename df columns and create metadata
    
    df_list.append(chunk)
    
df = pd.concat(df_list, ignore_index=True)
df.head()

,Sample Name,CAS9-A,CAS9-A.1,CAS9-B,CAS9-B.1,CAV_A,CAV_A.1,CAV_B,CAV_B.1,CAVIN_A,...,SPTLC_B,SPTLC_B.1,UGCG-A,UGCG-A.1,UGCG-B,UGCG-B.1,WT_A,WT_A.1,WT_B,WT_B.1
0,Hex2Cer 26:3;2 (LCB 18:0;2-2H2O),167.0000,143.0000,125.0000,132.0000,171.6091,182.9007,120.0000,120.0000,281.3523,...,130.0000,142.0000,68.0529,46.4729,88.0000,91.0000,229.5233,278.1255,206.0000,182.0000
1,Hex2Cer 26:2;2 (LCB 18:0;2-2H2O),53.2158,52.8015,47.4908,76.2783,27.6563,23.0569,9.0712,24.0712,0.0000,...,0.0000,2.1176,47.2560,62.7063,11.1856,0.0000,10.8768,6.0672,0.0000,34.4747
2,Hex2Cer 26:2;2 (LCB 18:0;2-2H2O),167.8122,131.3388,220.0689,128.3439,342.9760,331.4596,268.9979,289.4925,295.0869,...,290.0353,223.3067,187.9922,219.2679,231.3212,219.2373,329.9421,313.2847,251.0406,226.8066
3,Hex2Cer 26:2;2 (LCB 18:0;2-H2O),54.0000,47.0000,39.0000,44.0000,74.0000,64.0000,84.5341,88.7361,70.0000,...,44.0000,37.0000,48.6404,51.9843,38.9995,52.0000,62.0000,57.0000,51.0000,40.0000
4,Hex2Cer 26:0;2 (LCB 18:0;2-2H2O),391.2240,326.4641,444.7720,385.3202,175.0126,296.5953,238.6997,214.9442,215.1018,...,309.7805,272.9937,373.2111,292.2522,348.9607,348.3176,304.3199,271.4133,303.6544,253.3654


# Rename columns + get experiment metadata

In [3]:
# create list to hold rows for metadata
row_list = []

for name in df.columns[1:]:
    # split string to get protein
    p = re.split('-A|_A|-B|_B', name)
    # print(p[0])
        
    # create row for metadata
    row_list.append({'Exp': name, 'Mutation': p[0]})

# create metada
df_exps = pd.DataFrame(row_list)

In [4]:
# check exp metadata
df_exps.sample(3)

,Exp,Mutation
27,Flot2-B.1,Flot2
40,WT_B,WT
32,SPTLC_B,SPTLC


# Get Lipid Metadata

In [5]:
row_list = []

for name in df["Sample Name"]:
    # print(name)
    
    # split sample name string
    qual = re.split(' |:|;', name)
    # print(qual)
    
    # get head group, chain length, unsaturation
    head_group = qual[0]
    
    # get chain length
    chain_length = qual[1]
    if "-" in chain_length:
        c = chain_length.split(sep="-")
        chain_length = c[1]
        head_group += " " + c[0]
    chain_length = int(chain_length)
    
    # get unsaturation
    unsaturation = qual[2]
    if "+" in unsaturation:
        u = unsaturation.split(sep="+")
        unsaturation = u[0] 
    unsaturation = int(unsaturation)
    
    # create dict for row and then add to list of rows if not already in there
    row = {"Sample Name":name, 
           "Head Group":head_group, 
           "Acyl Chain Length":chain_length, 
           "Unsaturation":unsaturation}
    if row not in row_list:
        row_list.append(row)
    
    
df_meta = pd.DataFrame(row_list)
df_meta.sample(10)
# df_meta[df_meta["Sample Name"] == "DAG 29:3+NH4 (-FA 12:1 (NH4))"]

['Hex2Cer', '26', '3', '2', '(LCB', '18', '0', '2-2H2O)']
['Hex2Cer', '26', '2', '2', '(LCB', '18', '0', '2-2H2O)']
['Hex2Cer', '26', '2', '2', '(LCB', '18', '0', '2-2H2O)']
['Hex2Cer', '26', '2', '2', '(LCB', '18', '0', '2-H2O)']
['Hex2Cer', '26', '0', '2', '(LCB', '18', '0', '2-2H2O)']
['Hex2Cer', '26', '0', '2', '(LCB', '18', '0', '2-H2O)']
['Hex2Cer', '26', '4', '3', '(LCB', '18', '0', '2-H2O)']
['Hex2Cer', '26', '4', '3', '(LCB', '18', '1', '2-H2O,LCB', '18', '0', '3-2H2O)']
['Hex2Cer', '26', '3', '3', '(LCB', '18', '1', '2-H2O,LCB', '18', '0', '3-2H2O)']
['Hex2Cer', '26', '3', '3', '(LCB', '18', '1', '2-H2O,LCB', '18', '0', '3-2H2O)']
['Hex2Cer', '26', '3', '3', '(LCB', '18', '0', '2-H2O)']
['Hex2Cer', '26', '2', '3', '(LCB', '18', '0', '2-H2O)']
['Hex2Cer', '26', '0', '3', '(LCB', '18', '0', '2-2H2O)']
['Hex2Cer', '28', '2', '2', '(LCB', '18', '0', '2-H2O)']
['Hex2Cer', '28', '0', '2', '(LCB', '18', '0', '2-H2O)']
['Hex2Cer', '26', '1', '4', '(LCB', '17', '0', '2-2H2O)']
['Hex2C

['NAPE', '32', '7+NH4', '(-NAPE', '(20', '4', 'NH4))']
['NAPE', '34', '11+NH4', '(-NAPE', '(18', '1', 'NH4))']
['NAPE', '34', '6+NH4', '(-NAPE', '(22', '6', 'NH4))']
['NAPE', '34', '6+NH4', '(-NAPE', '(20', '4', 'NH4))']
['PE', 'O-38', '10', '(LPE', 'p)']
['NAPE', '34', '4+NH4', '(-NAPE', '(18', '1', 'NH4))']
['NAPE', '38', '12+NH4', '(-NAPE', '(18', '1', 'NH4))']
['NAPE', '38', '10+NH4', '(-NAPE', '(22', '5', 'NH4))']
['PE', '40', '1', '(LPE', 'p)']
['NAPE', '40', '3+NH4', '(-FA', '14', '0', '(NH4))']
['NAPE', '40', '0+NH4', '(-NAPE', '(22', '5', 'NH4))']
['NAPE', '46', '7+NH4', '(-FA', '14', '3', '(NH4))']
['LPE', 'O-20', '2', '(LPE', 'p)']
['NAPE', '30', '4+NH4', '(-NAPE', '(18', '0', 'NH4))']
['NAPE', '32', '9+NH4', '(-NAPE', '(18', '1', 'NH4))']
['NAPE', '32', '1+NH4', '(-NAPE', '(18', '2', 'NH4))']
['PE', 'O-36', '2', '(LPE', 'p)']
['NAPE', '34', '9+NH4', '(-NAPE', '(22', '5', 'NH4))']
['NAPE', '34', '8+NH4', '(-NAPE', '(22', '6', 'NH4))']
['NAPE', '34', '8+NH4', '(-NAPE', '(22',

['LPC', '20', '6', '(PC)']
['PC', '22', '2', '(PC)']
['LPC', '26', '4', '(PC)']
['PC', 'O-32', '2', '(PC', '104)']
['LPC', 'O-18', '4', '(LPC)']
['PC', 'O-36', '0', '(PC', '104)']
['PC', '38', '0', '(PC', '104)']
['LPC', 'O-20', '1', '(LPC)']
['LPC', 'O-28', '0', '(PC)']
['PC', 'O-20', '5', '(PC)']
['PC', '34', '0', '(PC', '104)']
['PC', 'O-44', '12', '(PC)']
['PC', '42', '0', '(LPC)']
['Hex3Cer', '34', '0', '3', '(LCB', '18', '0', '2-2H2O)']
['Hex3Cer', '34', '0', '3', '(LCB', '18', '0', '2-H2O)']
['Hex3Cer', '38', '3', '2', '(LCB', '18', '1', '2-2H2O,LCB', '18', '0', '3-3H2O)']
['Hex3Cer', '38', '3', '2', '(LCB', '18', '1', '2-H2O,LCB', '18', '0', '3-2H2O)']
['Hex3Cer', '38', '3', '2', '(LCB', '18', '0', '2-H2O)']
['Hex3Cer', '38', '3', '2', '(LCB', '17', '0', '2-2H2O)']
['Hex3Cer', '44', '0', '2', '(LCB', '18', '0', '2-H2O)']
['Hex3Cer', '46', '3', '4', '(LCB', '18', '0', '2-2H2O)']
['Hex3Cer', '46', '3', '4', '(LCB', '18', '1', '2-H2O,LCB', '18', '0', '3-2H2O)']
['Hex3Cer', '48', '

['GM2', '42', '0', '2', '(LCB', '18', '0', '2-H2O)']
['GM1', '30', '4', '3', '(LCB', '18', '0', '2-H2O)']
['GM2', '40', '0', '4', '(LCB', '18', '1', '2-2H2O,LCB', '18', '0', '3-3H2O)']
['GM1', '30', '2', '3', '(LCB', '18', '0', '2-2H2O)']
['GM1', '32', '1', '2', '(LCB', '18', '0', '2-2H2O)']
['GM1', '32', '0', '2', '(LCB', '18', '0', '2-2H2O)']
['GM2', '44', '1', '2', '(LCB', '18', '0', '2-2H2O)']
['GM2', '44', '1', '2', '(LCB', '18', '0', '2-H2O)']
['GM1', '32', '2', '3', '(LCB', '18', '0', '2-2H2O)']
['GM1', '32', '2', '3', '(LCB', '18', '1', '2-H2O,LCB', '18', '0', '3-2H2O)']
['GM1', '32', '2', '3', '(LCB', '18', '0', '2-H2O)']
['GM1', '32', '1', '3', '(LCB', '18', '0', '2-H2O)']
['GM2', '44', '1', '3', '(LCB', '18', '1', '2-H2O,LCB', '18', '0', '3-2H2O)']
['GM2', '44', '1', '3', '(LCB', '18', '0', '2-H2O)']
['GM1', '34', '1', '2', '(LCB', '18', '0', '2-H2O)']
['GM1', '34', '0', '2', '(LCB', '18', '0', '2-2H2O)']
['GM1', '34', '1', '2', '(LCB', '18', '1', '2-H2O,LCB', '18', '0', '3-

['TAG', '40', '5+NH4', '(-FA', '17', '1', '(NH4))']
['TAG', '40', '5+NH4', '(-FA', '17', '2', '(NH4))']
['TAG', '40', '5+NH4', '(-FA', '17', '3', '(NH4))']
['TAG', '40', '5+NH4', '(-FA', '18', '1', '(NH4))']
['TAG', '40', '5+NH4', '(-FA', '19', '1', '(NH4))']
['TAG', '40', '5+NH4', '(-FA', '19', '2', '(NH4))']
['TAG', '40', '5+NH4', '(-FA', '19', '3', '(NH4))']
['TAG', '40', '5+NH4', '(-FA', '20', '0', '(NH4))']
['TAG', '44', '5+NH4', '(-FA', '14', '3', '(NH4))']
['TAG', '44', '5+NH4', '(-FA', '15', '1', '(NH4))']
['TAG', '44', '5+NH4', '(-FA', '15', '2', '(NH4))']
['TAG', '44', '5+NH4', '(-FA', '16', '2', '(NH4))']
['TAG', '44', '5+NH4', '(-FA', '17', '2', '(NH4))']
['TAG', '44', '5+NH4', '(-FA', '18', '2', '(NH4))']
['TAG', '44', '5+NH4', '(-FA', '19', '1', '(NH4))']
['TAG', '44', '5+NH4', '(-FA', '20', '0', '(NH4))']
['TAG', '44', '5+NH4', '(-FA', '20', '2', '(NH4))']
['TAG', '44', '5+NH4', '(-FA', '24', '0', '(NH4))']
['TAG', '44', '4+NH4', '(-FA', '12', '2', '(NH4))']
['TAG', '44'

,Sample Name,Head Group,Acyl Chain Length,Unsaturation
4681,TAG 44:4+NH4 (-FA 16:2 (NH4)),TAG,44,4
2931,Hex3Cer 48:0;3 (LCB 18:0;2-2H2O),Hex3Cer,48,0
3417,HexCer 30:4;2 (LCB 18:0;2-2H2O),HexCer,30,4
3936,"GM3 40:0;3 (LCB 18:1;2-H2O,LCB 18:0;3-2H2O)",GM3,40,0
5301,MADAG 54:5+NH4 (-FA 15:1 (NH4)),MADAG,54,5
527,"GD3 30:4;2 (LCB 18:1;2-H2O,LCB 18:0;3-2H2O)",GD3,30,4
5260,MADAG 50:2+NH4 (-FA 15:2 (NH4)),MADAG,50,2
5114,MADAG 56:7+NH4 (-FA 20:5 (NH4)),MADAG,56,7
3165,"SGalCer 35:0;4 (LCB 18:1;2-H2O,LCB 18:0;3-2H2O)",SGalCer,35,0
2300,PS O-40:5 (FA 20:3),PS O,40,5


# Get Head Group Metadata

In [6]:
# list of original head groups
hg_list = df_meta['Head Group'].unique()

# list of head groups metadata
hg2_list = []

for hg in hg_list:
    # first sort the O groups (ex: PC, PC O)
    if " " in hg:
        hg2 = hg.split(" ")[0] 
        
    # sort the 1/2/3 groups(GD, GT)
    elif hg[-1] in ['1', '2', '3']:
        hg2 = hg[:-1]
    
    # get the hexcer
    elif 'Hex' in hg:
        hg2 = 'Hex_Cer'
        
    # get the acylglycerols
    elif hg in 'DAG,TAG,MAG':
        hg2='DAG,TAG,MAG'
    
    # all others    
    else:
        hg2 = hg
        
    hg2_list.append(hg2)
    
df_hg = pd.DataFrame({'Head Group': hg_list, 'Head Group 2': hg2_list})
df_hg.sort_values('Head Group').head(3)

,Head Group,Head Group 2
36,DAG,"DAG,TAG,MAG"
16,DMPE,DMPE
14,DMPE O,DMPE


# Get Unsaturation Metadata
Merge lipid metadata with head group and unsaturation metadata

In [7]:
import numpy as np

In [8]:
# merge df_meta with df_hg
df_meta2 = df_meta.merge(df_hg, on='Head Group')

# add unsaturation metadata
df_meta2['Unsaturation 2'] = np.where(df_meta2['Unsaturation'] < 3, df_meta2['Unsaturation'], '>=3')
df_meta2.head()

,Sample Name,Head Group,Acyl Chain Length,Unsaturation,Head Group 2,Unsaturation 2
0,Hex2Cer 26:3;2 (LCB 18:0;2-2H2O),Hex2Cer,26,3,Hex_Cer,>=3
1,Hex2Cer 26:2;2 (LCB 18:0;2-2H2O),Hex2Cer,26,2,Hex_Cer,2
2,Hex2Cer 26:2;2 (LCB 18:0;2-H2O),Hex2Cer,26,2,Hex_Cer,2
3,Hex2Cer 26:0;2 (LCB 18:0;2-2H2O),Hex2Cer,26,0,Hex_Cer,0
4,Hex2Cer 26:0;2 (LCB 18:0;2-H2O),Hex2Cer,26,0,Hex_Cer,0


# Save DataFrames

In [9]:
# save raw data (df), without index
df.to_csv('dataframes/pos_df.csv', index=False)

# save lipid metadata
df_meta.to_csv('dataframes/pos_lipids_df_meta.csv', index=False)

# save experiment metadata
df_exps.to_csv('dataframes/pos_df_exps.csv', index = False)

# save head group metadata
df_hg.to_csv('dataframes/pos_df_hg.csv', index = False)

# save lipid metadata, with combined grouping
df_meta2.to_csv('dataframes/pos_lipids_df_meta2.csv', index=False)